In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pano_generator
import os

c:\Users\ual\.conda\envs\felix\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tensorflow.keras.layers as layers

# some img_augmentation
img_augmentation = tf.keras.models.Sequential(
    [
        tf.keras.layers.RandomRotation(factor=0.15),
        tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        tf.keras.layers.RandomFlip(mode="horizontal"),
        tf.keras.layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

def transfer_model(number_classes,architecture = tf.keras.applications.efficientnet.EfficientNetB2,name=None):

    inputs = tf.keras.Input([260,260,3])

    x = img_augmentation(inputs) # apply image augmentaion

    model = architecture(
        include_top=False,
        weights='imagenet',
        input_tensor= x)

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(number_classes, activation="softmax", name="pred",kernel_regularizer=tf.keras.regularizers.L2())(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name=name)

    return model


def unfreeze_model(model,n_layers = 20):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-n_layers:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True


    return model

In [3]:
model = transfer_model(2,tf.keras.applications.MobileNetV3Small)
model._name = 'mv3'
model.compile(tf.keras.optimizers.Adam(1e-4),loss = tf.keras.losses.BinaryCrossentropy(),metrics=["accuracy"])

4334752/4334752 [==============================] - 0s 0us/step


In [4]:
log_dir =  "training_results/logs/" + model.name + "/"
checkpoint_path =  "training_results/weights/" + model.name + "/"
last_model = "training_results/last_model/" + model.name + "/"

try:
    os.makedirs(log_dir)
    os.makedirs(checkpoint_path)
    os.makedirs(last_model)
except:
    pass


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    monitor='val_accuracy',
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_best_only=True)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    verbose=1,
    mode='auto') 

In [5]:
train_ds , val_ds = pano_generator.get_train_and_val()

In [6]:
with tf.device('/device:GPU:0'):
    model.fit(train_ds,epochs=20,validation_data=val_ds,callbacks=[tensorboard_callback,cp_callback,early_stopping_callback])

Epoch 1/20
515/515 [==============================] - ETA: 0s - loss: 0.3298 - accuracy: 0.9220
Epoch 1: val_accuracy improved from -inf to 0.86458, saving model to training_results/weights/mv3\
515/515 [==============================] - 422s 804ms/step - loss: 0.3298 - accuracy: 0.9220 - val_loss: 0.3851 - val_accuracy: 0.8646
Epoch 2/20
 85/515 [===>..........................] - ETA: 5:47 - loss: 0.1576 - accuracy: 0.9833